In [2]:
import json
with open("data/train.json", 'r', encoding='utf-8') as f:
    train_data = json.load(f)

In [3]:
for ind, data in train_data.items():
    query = data['query']
    if 'document' in data:
        document = data['document']
        if len(document) == 0:
            break

In [4]:
data

{'query': 'a#HASH# chord',
 'facet': ['guitar', 'piano', 'ukulele'],
 'document': [],
 'related': []}

In [5]:
train_data['400']

{'query': '100 best weight watchers recipes',
 'facet': ['beef',
  'butternut squash',
  'spaghetti squash',
  'broccoli',
  'broccoli'],
 'document': ['We have over 100 delicious Weight Watchers recipes with their WW SmartPoints. Find quick and easy WW recipes broken up by their points (zero on up) for instant pot, crock pot, chicken, beef and more.',
  'WWs most popular recipes Check out the 20 best recipes that our members come back to time and again. Go-to dishes are a big part of a successful meal plan because you know how to make them and how many SmartPoints they have.',
  '50 Favorite Weight Watchers Recipes. Whether youre craving a creamy casserole or chocolate fudge, get ready to indulge with these popular guilt-free Weight Watchers recipes.',
  'Use the dropdown box below to sort every recipe on my blog by its WW Smart Points total. You can also use the roundups below in case you’re looking for a more specific search: 22 Weight Watchers Chicken Recipes with 5 Smart Points or

In [6]:
# import re
# text = ""
# for sinppet in train_data['400']['document']:
#     text += sinppet + '\n' 
# text = text.strip()

# def remove_tag(content):
#     cleanr =re.compile('<.*?>')
#     cleantext = re.sub(cleanr, '', content)
#     return cleantext

# def cleanhtml(raw_html):
#     cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
#     cleantext = re.sub(cleanr, '', raw_html)
#     return cleantext

# # re.sub('<[^<]+?>', '', text)
# print(cleanhtml(text))

In [7]:
# print(remove_tag(text))

In [8]:
import json
with open("data/test.json", 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [9]:
test_data['0']

{'query': 'caesars atlantic city',
 'facet': ['caesars atlantic city events',
  'caesars atlantic city jobs',
  'caesars atlantic city parking'],
 'options_overall_label': 1,
 'document': ['Caesars Atlantic City Disappointed, frustrated, miserable Worst hotel experience My kids wanted to experience Atlantic City so book Caesar’s base on advertising it was a nightmare for my kids, room and bathroom nasty, holes in rags and towels dingy like been using for the past...',
  'Now $55 (Was $̶1̶1̶4̶) on Tripadvisor: Caesars Atlantic City, Atlantic City. See 18,311 traveler reviews, 1,818 candid photos, and great deals for Caesars Atlantic City, ranked #3 of 50 hotels in Atlantic City and rated 4 of 5 at Tripadvisor.',
  'Book the Caesars Atlantic City Resort  Casino - Stay at this 4-star beach resort in Atlantic City. Enjoy free WiFi, an outdoor pool, and 10 restaurants. Our guests praise the restaurant and the comfy rooms in our reviews. Popular attractions Ballys Atlantic City Casino and Bo

In [10]:
from transformers import AutoTokenizer, BartForConditionalGeneration

# model_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/bart-base"
model_path = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path) # facebook/bart-base

In [11]:
input_string = "caesars atlantic city"
enc_input = tokenizer(input_string, max_length=1024, return_tensors="pt")

taget_string = "caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking"
dec_input = tokenizer(taget_string, max_length=1024, return_tensors="pt")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
enc_input_ids = enc_input['input_ids']
enc_attention_mask = enc_input['attention_mask']

dec_input_ids = dec_input['input_ids']
labels = dec_input_ids[:, :-1]
dec_input_ids = dec_input_ids[:, 1:]
dec_attention_mask = dec_input['attention_mask'][:,1:]

In [15]:
dec_input_ids

tensor([[ 3245,   293,  2726,    23,   462, 26970,   343,  1061,     6,  6056,
           293,  2726,    23,   462, 26970,   343,  1315,     6,  6056,   293,
          2726,    23,   462, 26970,   343,  2932,     2]])

In [20]:
tokenizer.decode(enc_input_ids[0]), tokenizer.decode(dec_input_ids[0]), tokenizer.decode(labels[0])

('<s>caesars atlantic city</s>',
 'caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking</s>',
 '<s>caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking')

In [21]:
outputs = model(
    input_ids=enc_input_ids,
    attention_mask=enc_attention_mask,
    decoder_input_ids=dec_input_ids,
    decoder_attention_mask=dec_attention_mask,
    labels=labels
)

In [4]:
output = model(inputs['input_ids'], labels=inputs["input_ids"])

In [5]:
output.keys()

odict_keys(['loss', 'logits', 'encoder_last_hidden_state'])

In [6]:
output['logits'].shape

torch.Size([1, 36, 50265])

In [30]:
output['loss']

tensor(0.0323, grad_fn=<NllLossBackward0>)

In [32]:
test_string = "caesars atlantic city:"
inputs = tokenizer([test_string], max_length=1024, return_tensors="pt")

gen_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
tokenizer.batch_decode(gen_ids, skip_special_tokens=True)[0]

'caesars atlantic city:'